In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
data = pd.read_csv("/kaggle/input/titanic/train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Prints a statistical outline of the data
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


The probability of a woman or man surviving is
$$
    \begin{align*}
        P(F) = 0.35...\\
        P(M) = 0.64... \\
        P(S|F) = 0.74...\\
        P(D|M) = 0.81...\\
    \end{align*}
$$
Our model should be at least more accurate than simply guessing all women survive and all men die

$$
    \begin{align*}
        Acc &= P(S|F)*P(F) + P(D|M)*P(M)\\
        Acc &= 0.74 * 0.35 + 0.81 * 0.64\\
        Acc &= 0.78...
    \end{align*}
$$

In [4]:
## Calculating P(F) P(M) P(S|F) and P(D|M)
females = data.loc[data["Sex"] == 'female']
female_survivors = females.loc[females["Survived"] == 1]
female_survival_rate = len(female_survivors) / len(females)
female_ratio = len(females) / len(data)

males = data.loc[data["Sex"] == 'male']
male_survivors = males.loc[males["Survived"] == 1]
male_survival_rate = len(male_survivors) / len(males)
male_ratio = len(males) / len(data)

## Calculating the accuracy of this model
sex_model_accuracy = female_survival_rate * female_ratio + (1 - male_survival_rate) * male_ratio

print("Survival Chance by Gender")
print(f"P(F): {female_ratio} P(S|F): {female_survival_rate}")   
print(f"P(M): {male_ratio} P(D|M): {1 - male_survival_rate}")
print(f"Sex Model accuracy = {sex_model_accuracy}")

Survival Chance by Gender
P(F): 0.35241301907968575 P(S|F): 0.7420382165605095
P(M): 0.6475869809203143 P(D|M): 0.8110918544194108
Sex Model accuracy = 0.7867564534231202


Next lets see if we can improve the model using a random forest classifier

In [5]:
from sklearn.ensemble import RandomForestClassifier

y = data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(data[features])
X_test = pd.get_dummies(test[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
